# Homework 4

These are the libraries we use.

In [71]:
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, regexp_tokenize
#from selenium import webdriver
import time
import math
from sklearn.cluster import KMeans
import re
import time
import warnings
warnings.filterwarnings("ignore")

## 1) Does basic house information reflect house's description?

We use **Beautiful soup** to retrieve about $10K$ announcements from the site **immobiliare.it**. We want to build two matrices: the first one $m_{ij}$ with {prezzo, locali, superficie, bagni, piano} for each announcement and the second one with the Tfidf value of each word in each document. In addition to this we need the entire description of each announcement.

In [ ]:
%%time
records = []
count = 1
start_time = time.time()
for i in tqdm(range(1, 1700)):
    r = requests.get('https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=' + str(i+1), verify = False)  
    soup = BeautifulSoup(r.text, 'html.parser')

    for block in soup.find_all('div', class_ = 'listing-item_body'):
        try:
            list_ = block.find('ul', class_ = 'listing-features list-piped')
            if len(list_) == 7:
                title = 'announcement_' + str(count)
                items = block.find_all('li')
                prezzo = items[0].text.strip()[2:]
                prezzo = prezzo.replace('.', '')
                locali = items[1].text[0]
                superfici = items[2].text[0:-12]
                bagni = items[3].text[0:-5]
                piano = items[4].text[0:-7].strip()
             
                url = block.find('a', href = True)

                if ('https://' in url['href']) == True:
                    ann = requests.get(url['href'])
                else:
                    ann = requests.get('https://www.immobiliare.it' + url['href'])

                data = ann.text
                soup1 = BeautifulSoup(data, 'html.parser')
                a = soup1.find('div', attrs = {'role':'contentinfo'}).text.strip()


                records.append((title, prezzo, locali, superfici, bagni, piano, a))
                count += 1
        except:
            pass
            

    time.sleep(1)
    
print(time.time() - start_time)

In [ ]:
len(records)

In [3]:
%%time
records = []
count = 1
start_time = time.time()
for i in tqdm(range(1,1001)):
    r = requests.get('https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=' + str(i), verify = False)  
    soup = BeautifulSoup(r.text, 'html.parser')

    for block in soup.find_all('div', class_ = 'listing-item_body'):
        try:
            list_ = block.find('ul', class_ = 'listing-features list-piped')
            #if len(list_) == 7:
            title = 'announcement_' + str(count)
                #print('title: ', title)
            prezzo_ = block.findAll('li', class_ = 'lif__item lif__pricing')
            prezzo = prezzo_[-1].text.strip()
                #print('\nprezzo ', prezzo)
            locali_ = block.findAll('span', class_ = 'text-bold')
            locali = locali_[-3].text
                #print('\nlocali ', locali)
            superfici = locali_[-2].text
                #print('\nsuperfici ', superfici)
            bagni = locali_[-1].text
                #print('\nbagni ', bagni)
            piano = block.div.abbr['title']
                #print('\npiano ', piano)
            if (piano == 'T' or piano == 'Piano terra'): 
                piano = 0
            elif (piano.isdigit() == False or prezzo == 'EZZO SU RICHIESTA'):
                continue
            if ('%' in prezzo):
                prezzo = '€' + prezzo.split('€')[1]
                
            url = block.find('a', href = True)

            if ('https://' in url['href']):
                ann = requests.get(url['href'])
            else:
                ann = requests.get('https://www.immobiliare.it' + url['href'])

            data = ann.text
            soup1 = BeautifulSoup(data, 'html.parser')

            desc = soup1.find('div', attrs = {'role':'contentinfo'}).text.strip()
                    #print('desc  ----- ', desc)


            records.append((title, prezzo, locali, superfici, bagni, piano, desc))
            count += 1
                        
        except:
            pass
            

    time.sleep(1)
    
print(time.time() - start_time)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [3:41:32<00:00, 12.87s/it]


13292.137946128845
Wall time: 3h 41min 32s


In [4]:
len(records)

16148

Creating a df from the announcements we get

In [73]:
df = pd.DataFrame(records, columns = ['annuncio', 'prezzo', 'locali', 'superfici', 'bagni', 'piano', 'descrizione'])
df.replace({r'\+': '', r'\n': '',  r'\xa0' : ''}, regex = True)

,annuncio,prezzo,locali,superfici,bagni,piano,descrizione
0,announcement_1,€ 225.000,2,50,1,1,PAPILLO EUR in elegante complesso residenziale...
1,announcement_2,€ 375.000,3,120,1,3,"Pigneto, Appartamento in Vendita, libero lumin..."
2,announcement_3,€ 68.000,2,20,1,0,Rif: 18066 - TRIESTE - Via della Fontana. Nell...
3,announcement_4,€ 450.000,5,150,2,3,Zona EUR - Viale Cesare Pavese - Via Salvatore...
4,announcement_5,€ 315.000,3,92,2,5,Fino al 31/12/2018 le spese relative l’onorari...
5,announcement_6,€ 579.000,5,160,2,2,"Eur, Appartamento in vendita via Cesare Pavese..."
6,announcement_7,€ 385.000,3,100,3,2,"San Giovanni, Appartamento in Vendita, adiacen..."
7,announcement_8,€ 339.000,3,90,1,4,"San Giovanni, Appartamento in Vendita, finemen..."
8,announcement_9,€ 215.000,3,67,1,1,Rif: 18105 - PORTA DI ROMA - Via Carlo Dapport...
9,announcement_10,€ 670.000,5,227,3,3,Nelle immediate vicinanze di viale Cortina D’A...


Replacing some oddities within the df column

In [74]:
df['locali'] = df['locali'].str.replace('\xa0', '')
df['locali'] = df['locali'].str.replace('+', '')
df['bagni'] = df['bagni'].str.replace('\xa0', '')
df['bagni'] = df['bagni'].str.replace('+', '')
df['superfici'] = df['superfici'].str.replace('\xa0', '')
df['superfici'] = df['superfici'].str.replace('.', '')
df['piano'] = df['piano'].str.replace('\xa0', '')

In [81]:
# number of NaN in the column 'piano'
df['piano'].isna().sum()

2761

In [83]:
# replacing NaN value with 0
df['piano'] = df['piano'].fillna(0)

In [85]:
# converting the columns items into integers
df['locali'] = df['locali'].astype(np.int64)
df['superfici'] = df['superfici'].astype(np.int64)
df['bagni'] = df['bagni'].astype(np.int64)
df['piano'] = df['piano'].astype(np.int64)

In [86]:
df

,annuncio,prezzo,locali,superfici,bagni,piano,descrizione
0,announcement_1,€ 225.000,2,50,1,1,PAPILLO EUR in elegante complesso residenziale...
1,announcement_2,€ 375.000,3,120,1,3,"Pigneto, Appartamento in Vendita, libero lumin..."
2,announcement_3,€ 68.000,2,20,1,0,Rif: 18066 - TRIESTE - Via della Fontana. Nell...
3,announcement_4,€ 450.000,5,150,2,3,Zona EUR - Viale Cesare Pavese - Via Salvatore...
4,announcement_5,€ 315.000,3,92,2,5,Fino al 31/12/2018 le spese relative l’onorari...
5,announcement_6,€ 579.000,5,160,2,2,"Eur, Appartamento in vendita via Cesare Pavese..."
6,announcement_7,€ 385.000,3,100,3,2,"San Giovanni, Appartamento in Vendita, adiacen..."
7,announcement_8,€ 339.000,3,90,1,4,"San Giovanni, Appartamento in Vendita, finemen..."
8,announcement_9,€ 215.000,3,67,1,1,Rif: 18105 - PORTA DI ROMA - Via Carlo Dapport...
9,announcement_10,€ 670.000,5,227,3,3,Nelle immediate vicinanze di viale Cortina D’A...


We save the first dataframe into a .csv file 

In [87]:
df.to_csv('items_df.csv', sep = '\t', encoding = 'utf-8')

Function to preprocess the documents

In [88]:
def preprocess(text):
    text = text.lower()
    # removing '\n'
    text = text.replace('\\n', ' ')
    # removing punctuation
    tokenizer = regexp_tokenize(text, "[\w\$]+")
    # filter the non stopwords
    filtered = [w for w in tokenizer if not w in stopwords.words('italian')]
    ps = PorterStemmer()
    # removing the stem
    filtered = [ps.stem(word) for word in filtered]
    return filtered

In [ ]:
# processed is a nested list 
# each item is a list of preprocessed words of each announcement without repetitions
processed = []
for announcement in tqdm(records):
    processed.append(list(set(preprocess(announcement[-1]))))

 17%|█████████████                                                                | 2728/16148 [03:23<17:29, 12.79it/s]

In [ ]:
# flatten list
tot_proc = []
for li in processed:
    for el in li:
        tot_proc.append(el)
# words without repetitions
tot_unique = list(set(tot_proc))

Creating the tf_idf for each word

In [ ]:
# counting the occurrencies in all the documents
def tfidf(doc, tot_announc, tot_proc, word):
    tfidf = doc.count(word)/len(doc) * math.log(tot_announc / tot_proc.count(word))
    return tfidf

In [ ]:
# number of announcements
tot_announc

In [ ]:
# it returns a list of tfidf (in each document) for a given word
def tfidf(processed, tot_announc, tot_proc, word):
    tfidf_li = []
    for i in range(len(processed)):
        tfidf = processed[i].count(word)/len(processed[i]) * math.log(tot_announc / tot_proc.count(word))
        tfidf_li.append(round(tfidf,4))
    return tfidf_li

In [ ]:
# dict {word: [tfidf_doc1, tfidf_doc1, ...]}
mydict = {}
for word in tot_unique:
    li = tfidf(processed, tot_announc, tot_proc, word)
    mydict[word] = li

In [ ]:
mydict

In [ ]:
announc_name = []
for i in range(1,16):
    announc_name.append('announcement_' + str(i))

In [ ]:
tfidf_df = pd.DataFrame(mydict)

In [ ]:
tfidf_df.index = announc_name

In [ ]:
tfidf_df

In [ ]:
tfidf_df.to_csv('items_tfidf_df.csv', sep = '\t', encoding = 'utf-8')

In [ ]:
# removing the + sign from the column 'bagni'
df = df.drop(labels = 'descrizione', axis = 1)
df = df.drop(labels = 'annuncio', axis = 1)
df

In [ ]:
# converting df and tfidf_df to matrices
df_matrix = df.values
tfidf_matrix = tfidf_df.values
df_matrix

In [ ]:
#second_mat.to_csv(r'C:\Users\Daniele\Desktop\Matrix_TfIdf.csv')
#table.to_csv(r'C:\Users\Daniele\Desktop\Matrix.csv')

In [ ]:
km = KMeans(n_clusters = 5, init = 'k-means++').fit(df_matrix)
km1 = KMeans(n_clusters = 5, init = 'k-means++').fit(tfidf_matrix)

In [ ]:
print(km.cluster_centers_)
print(km1.cluster_centers_)

## 2) Find the duplicates!

In [ ]:
a =  open('passwords2.txt', 'r')
data = a.read()
first = data.split('\n', 1)[0]

In [ ]:
first

In [ ]:
hash(first)